## Pure LLM

In [3]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]  
)

messages1 = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the weather tomorrow?")
]

response = llm_model.invoke(messages1)
# print(response.content)

I can't fetch real-time weather data, but you can check reliable weather services like:

- **Weather.com** ([weather.com](https://weather.com))  
- **AccuWeather** ([accuweather.com](https://www.accuweather.com))  
- **The Weather Channel** ([weather.gov](https://www.weather.gov) for the U.S.)  

Let me know your location (city/country), and I can help interpret forecasts or provide general climate info! 🌤️

## Agent - 항공 호텔 예약 （LLM + Function calling）

In [2]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from typing import Annotated
import json
import requests
from langgraph.checkpoint.memory import InMemorySaver
import datetime
import pytz

MODEL_URL = 'https://api.deepseek.com'
MODEL_API_KEY = 'sk-7a3d4f4b2ca54d2dbb09e88d3572a242'

BASE_URL = "https://serpapi.com/search"
SERPER_API_KEY='39e13214c84bb203818c0f76ce4552fdb310538fd92a2026c93561dc54ff9bff'

def search_hotels(
    query: Annotated[str, "The city name or hotel name to search"], 
    check_in_date: Annotated[str, "Hotel Check-in date in YYYY-MM-DD format"], 
    check_out_date: Annotated[str, "Hotel Check-out date in YYYY-MM-DD format"],
) -> Annotated[str, "Returns a JSON string containing the search results."]:
    """
    Function to search hotels based on the given query, check-in date, and check-out date.
    """
    params = {
        "api_key": SERPER_API_KEY,
        "engine": "google_hotels",
        "q": query,
        "check_in_date": check_in_date,
        "check_out_date": check_out_date,
        "currency": "KRW",
        "hl": "ko",
        "gl": "kr",
    }
    # print("Calling search_hotels with params:", {"query": query, "check_in_date": check_in_date, "check_out_date": check_out_date})
    print("Calling search_hotels with params:", params)
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    print(response)
    return json.dumps(result) # llm에 보냄

def search_flights(
    departure_id: Annotated[str, "Departure id in IATA format"], 
    arrival_id: Annotated[str, "Arrival id in IATA format"],
    outbound_date: Annotated[str, "Flight outbound date in YYYY-MM-DD format"],
    return_date: Annotated[str, "Flight return date in YYYY-MM-DD format, pass empty string if it's a one-way flight"],
) -> Annotated[str, "Returns a JSON string containing the search results."]:
    """
    Function to search flights
    """
    params = {
        'api_key': SERPER_API_KEY,
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "currency": "KRW",
        "hl": "ko",
        "gl": "kr",
    }

    if return_date != "":
        params["return_date"] = return_date
    else:
        params["type"] = 2

    # print("Calling search_flights with params:", {"departure_id": departure_id, "arrival_id": arrival_id, "outbound_date": outbound_date, "return_date": return_date})
    print("Calling search_flights with params:", params)
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    print(response)
    return json.dumps(result)

def get_curent_datetime(
    zone: Annotated[str, "Timezone in pytz format to get the current datetime"],
) -> Annotated[str, "Returns the current datetime in '%Y-%m-%d %H:%M:%S %Z%z' format."]:
    """
    Function to get the current datetime in the given timezone.
    """
    print("Calling get_curent_datetime with timezone:", zone)
    local_timezone = pytz.timezone(zone)
    now = datetime.datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z%z")
    print("Current datetime in", zone, ":", now)
    return now

# system prompt
prompt = """
당신은 항공편과 호텔을 예약해주는 여행 예약 에이전트입니다. 사용자의 의도를 파악하고 필요한 API를 호출하여 예약 정보를 제공합니다.

행동 가이드라인:
- 항공편 예약 요청 시, 출발지와 도착지 이름을 IATA 공항 코드로 변환하세요.
- 호텔 또는 항공편 예약 시, 필요한 인자가 있다면 이를 추론하거나 기본값을 사용해 API를 호출하세요.
- 특정 지역의 현재 시간이나 날짜(예: 출발지의 내일 날짜 등)가 필요한 경우에는 get_curent_datetime API를 사용하세요.
- 호텔 또는 항공 예약이 아닌 일반 요청이라면 적절한 답변만 출력하세요.

출력 형식:
- 항공편은 **출발공항 | 항공사 | 편명 | 출발시간 | 도착공항 | 도착시간 | 소요시간 | 항공기종 | 좌석등급 | 가격 (KRW) | 다리공간 | 부가옵션 | 탄소배출량 (kg)** 순으로 마크다운 테이블로 출력합니다.
- 호텔은 **호텔명 | 호텔 설명 | 체크인 시간 | 체크아웃 시간 | 가격 (KRW) | 주변 장소 | 등급 | GPS 좌표** 순으로 마크다운 테이블로 출력합니다.

모든 질문과 답변은 한국어로 진행하세요.
"""

model = ChatOpenAI(
    model='deepseek-chat',
    base_url=MODEL_URL,
    api_key=MODEL_API_KEY,
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]  
)

checkpointer = InMemorySaver() # 채팅기록 저장

agent = create_react_agent(
    model=model,
    tools=[search_hotels, search_flights, get_curent_datetime],
    checkpointer=checkpointer, # 히스토리
    prompt=prompt, # 요구사항
)

while True:
    user_input = input("You: ")
    if user_input.lower() == "bye":
        break
    
    print(f"\n\nYou: \n{user_input}")
    print("\nAssistant: \n", end="") 
    
    # llm에게 메시를 보냄
    agent.invoke(
        {"messages": user_input},
        config={"configurable": {"thread_id": "1"}}
    )

# Example prompts:
# 1. 나는 내일 아침 서울에서 발리로 가고 싶어.
# 2. 나는 거기서 3일을 지낼거야. 가장 싼 호텔을 예약해줘.
# 3. 그냥 베스트 항공권이랑 호텔 보여줘. 너가 알아서 예약해줘.



You: 
나는 내일 아침 서울에서 발리로 가고 싶어.

Assistant: 
Calling get_curent_datetime with timezone: Asia/Seoul
Current datetime in Asia/Seoul : 2025-05-30 07:37:06 KST+0900
Calling search_flights with params: {'api_key': '39e13214c84bb203818c0f76ce4552fdb310538fd92a2026c93561dc54ff9bff', 'engine': 'google_flights', 'departure_id': 'ICN', 'arrival_id': 'DPS', 'outbound_date': '2025-05-31', 'currency': 'KRW', 'hl': 'ko', 'gl': 'kr', 'type': 2}
<Response [200]>
내일(2025년 5월 31일) 서울(인천국제공항, ICN)에서 발리(응우라라이 공항, DPS)로 가는 항공편 정보입니다. 아래는 추천 항공편입니다:

### 추천 항공편
| 출발공항 | 항공사 | 편명 | 출발시간 | 도착공항 | 도착시간 | 소요시간 | 항공기종 | 좌석등급 | 가격 (KRW) | 다리공간 | 부가옵션 | 탄소배출량 (kg) |
|----------|--------|------|----------|----------|----------|----------|-----------|-----------|------------|-----------|-----------|------------------|
| 인천국제공항 | 제주항공 | 7C 2711 | 15:40 | 응우라라이 공항 | 21:45 | 7시간 5분 | Boeing 737MAX 8 | Economy | 708,700 | 74cm | USB 포트, 탄소배출량: 338kg | 338 |
| 인천국제공항 | 대한항공 | KE 629 | 17:45 | 응우라라이 공항 | 23:35 | 6시간 50분 

## MCP

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]  
)

mcp_client = MultiServerMCPClient({
    "mcp_weather_server": {
        "url": "https://server.smithery.ai/@isdaniel/mcp_weather_server/mcp?api_key=e876ad75-848e-4182-a0d0-7e7343314dd6",
        "transport": "streamable_http",
    },
    "yahoo_finance_mcp": {
        "url": "https://server.smithery.ai/@hwangwoohyun-nav/yahoo-finance-mcp/mcp?api_key=e876ad75-848e-4182-a0d0-7e7343314dd6",
        "transport": "streamable_http",
    }
})
tools = await mcp_client.get_tools() # 자동으로 document를 가져옴
print("Tools: ", tools)

agent = create_react_agent(
    model=model,
    tools=tools
)

q = "What is the weather in Seoul?"
print("Q: ", q)
await agent.ainvoke({"messages": q})

print("\n\n========================\n\n")

q = "What is the current P/E ratio of Nvidia, and how does it compare to its 5-year average and to AMD?"
print("Q: ", q)
await agent.ainvoke({"messages": q})

Tools:  [StructuredTool(name='get_current_weather', description="Get current weather information for a specified city.\nIt extracts the current hour's temperature and weather code, maps\nthe weather code to a human-readable description, and returns a formatted summary.\n", args_schema={'properties': {'city': {'description': "The name of the city to fetch weather information for, PLEASE NOTE English name only, if the parameter city isn't English please translate to English before invoking this function.", 'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_current_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x12220ade0>), StructuredTool(name='get_weather_by_datetime_range', description='\nGet weather information for a specified city between start and end dates.\n', args_schema={'properties': {'city': {'description': "The name of the city to fetch weather in

{'messages': [HumanMessage(content='What is the current P/E ratio of Nvidia, and how does it compare to its 5-year average and to AMD?', additional_kwargs={}, response_metadata={}, id='7db1acff-a5b2-402f-af26-74e122aeacfd'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0_03df64d0-cfa9-45f8-8174-a88d4343ceaa', 'function': {'arguments': '{"ticker": "NVDA"}', 'name': 'get_stock_info'}, 'type': 'function'}, {'index': 1, 'id': 'call_1_721ddd8f-c04a-49b9-b709-ef9cdf49a89c', 'function': {'arguments': '{"ticker": "AMD"}', 'name': 'get_stock_info'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8'}, id='run--6f03e356-7e12-4254-8403-4365eb0b7c1f-0', tool_calls=[{'name': 'get_stock_info', 'args': {'ticker': 'NVDA'}, 'id': 'call_0_03df64d0-cfa9-45f8-8174-a88d4343ceaa', 'type': 'tool_call'}, {'name': 'get_stock_info', 'args': {'ticker': 'AMD'}, 'id': 'call_1